In [1]:
import pandas as pd
import geopandas as gpd
import re
import matplotlib as plt
import numpy as np

In [2]:
trash = pd.read_csv('cleaned_trash_haulers.csv')

In [3]:
trash['Date Opened'] = pd.to_datetime(trash['Date Opened'])

In [4]:
trash['Zip Code'] = trash['Zip Code'].astype('str')
trash['Zip Code'] = trash['Zip Code'].str.extract(r'(\d{5})')

In [5]:
trash.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20226 entries, 0 to 20225
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Request Number  20226 non-null  int64         
 1   Date Opened     20226 non-null  datetime64[ns]
 2   Request         20226 non-null  object        
 3   Description     20192 non-null  object        
 4   Zip Code        20186 non-null  object        
 5   Trash Hauler    19364 non-null  object        
 6   Trash Route     19279 non-null  object        
 7   LONGITUDE       20198 non-null  float64       
 8   LATITUDE        20198 non-null  float64       
 9   Clean Address   20216 non-null  object        
 10  Missed          20226 non-null  bool          
dtypes: bool(1), datetime64[ns](1), float64(2), int64(1), object(6)
memory usage: 1.6+ MB


# Red River Waste Solutions Total Damages
- For each address:
    - `>= 3 missed pickups on 3 unique dates`
    - `Within a 6 month timespan`
    + `+ $1500`

In [6]:
redriver = trash[['Request Number', 'Date Opened', 'Missed', 'Clean Address', 'Trash Hauler']]

In [8]:
redriver.head()

,Request Number,Date Opened,Missed,Clean Address,Trash Hauler
0,25270,2017-11-01,False,3817 CROUCH DR,RED RIVER
1,25274,2017-11-01,True,4028 CLARKSVILLE PIKE,RED RIVER
2,25276,2017-11-01,True,6528 THUNDERBIRD DR,RED RIVER
3,25307,2017-11-01,True,2603 OLD MATTHEWS RD,WASTE IND
4,25312,2017-11-01,True,604 CROLEY DR,RED RIVER


In [9]:
redriver = redriver.loc[redriver['Trash Hauler'] == 'RED RIVER']

In [10]:
redriver = redriver.loc[redriver['Missed'] == True]

In [11]:
redriver.head()

,Request Number,Date Opened,Missed,Clean Address,Trash Hauler
1,25274,2017-11-01,True,4028 CLARKSVILLE PIKE,RED RIVER
2,25276,2017-11-01,True,6528 THUNDERBIRD DR,RED RIVER
4,25312,2017-11-01,True,604 CROLEY DR,RED RIVER
8,25330,2017-11-01,True,4484 LAVERGNE COUCHVILLE PIKE,RED RIVER
10,25341,2017-11-01,True,3113 HYDES FERRY RD,RED RIVER


In [12]:
redriver = redriver.drop_duplicates(['Date Opened', 'Clean Address'])

In [13]:
#redriver['Time Diff'] = redriver.groupby('Clean Address')['Date Opened'].rolling(window = 3, min_periods = 3, center = True).DateOffset().reset_index(drop = True)

In [14]:
#redriver['timediff'] = redriver['Date Opened'].diff()

# Sort by date
# Make set of Addresses
- For loop for Address set
    - Make temporary df for address
    - For loop for dataframe
      - If firstdiff = date_opened - next row date opened
        - If first diff < 180 days
                - seconddiff = date_opened - 2nd next row date opened
                - If seconddiff < 180 days
                    - fine = True

In [15]:
redriver = redriver.sort_values('Date Opened').reset_index(drop = True)

In [16]:
address_set = set(redriver['Clean Address'])
address_set

{'1203 MCCHESNEY AVE',
 '408 ELYSIAN FIELDS RD',
 '5700 CLOVERLAND PARK PL',
 '4301 ELKINS AVE',
 '1728 DR D B TODD JR BLVD',
 '294 LE BON RD',
 '2701 HYDES FERRY RD',
 '223 CAPITOL VIEW AVE',
 '1110 RIVERSIDE DR',
 '4024 WOODMONT BLVD A',
 '3906 W VALLEY DR',
 '112 ROSEBANK AVE',
 '4303 LONE OAK RD',
 '703 HOLDER DR',
 '958 PERCY WARNER BLVD',
 '1218 HOWARD AVE',
 '3029 OWENDALE DR',
 '812 APPLE DR',
 '4724 AARON DR',
 '654 DELAWARE AVE',
 '324 WESTCHESTER DR',
 '4328 BRUSH HILL RD',
 '713 SUMMERFIELD DR',
 '4012 W HAMILTON RD',
 '1102 SNOW AVE',
 '941 BORDEAUX PL',
 '2313 PENNINGTON BEND RD',
 '3733 SEASONS DR',
 '2606 TATE CT',
 '212 VALLEY BEND DR',
 '3914 CREEKSIDE DR',
 '106 LEXINGTON CT',
 '3921 FLICKER DR',
 '2840 BUENA VISTA PIKE',
 '1714 HEIMAN ST',
 '412 CHARLESGATE CT',
 '2224 HAMLET HILL DRIVE',
 '2825 MOSSDALE DR',
 '2885 LERA JONES DR',
 '2924 MCGAVOCK PK',
 '635 OLD HICKORY BLVD',
 '960 BATTERY LN',
 '489 HOGAN RD',
 '3309 WESTCLIFFE CIR',
 '733 DUE WEST AVE N',
 '2509 

In [17]:
redriver['fine'] = False
#redriver['used_in_fine'] = False

In [18]:
for addy in address_set: # Iterates through each unique address
    addy_df = redriver.loc[redriver['Clean Address'] == addy].reset_index(drop = True) # Temp dataframe
    for index, fields in addy_df.iterrows():
        if len(addy_df) > 2: # Checks for at least 3 requests
          reqnum = fields['Request Number']
          if abs((addy_df.iloc[0, 1] - addy_df.iloc[1, 1]).days) <= 180:
              if abs((addy_df.iloc[0, 1] - addy_df.iloc[2, 1]).days) <= 180:
                  i = redriver[redriver['Request Number'] == reqnum].index
                  redriver.loc[i, 'fine'] = True
        addy_df = addy_df.drop(index=0).reset_index(drop = True)

In [19]:
redriver.head()

,Request Number,Date Opened,Missed,Clean Address,Trash Hauler,fine
0,25274,2017-11-01,True,4028 CLARKSVILLE PIKE,RED RIVER,False
1,25529,2017-11-01,True,131 PAGE RD,RED RIVER,False
2,25525,2017-11-01,True,162 ANTIOCH PIKE,RED RIVER,True
3,25522,2017-11-01,True,405 BROOK HOLLOW RD,RED RIVER,False
4,25519,2017-11-01,True,3702 HOBBS RD,RED RIVER,True


In [22]:
finetotal = 1500 * redriver['fine'].sum()
finetotal

2152500